In [1]:
%%writefile dashboard.py
import pymysql
import numpy as np
from PIL import Image
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from datetime import datetime
from datetime import datetime, timedelta 
from streamlit_extras.metric_cards import style_metric_cards
st.set_page_config(page_title="My App",layout='wide')
# Establish a database connection
data_base = pymysql.connect(host='localhost', user='root', passwd='Hiba1Adam2#')

# Create a cursor object
cur = data_base.cursor()

# Define the SQL query
query = 'SHOW DATABASES'

# Execute the query
cur.execute(query)

# Fetch the results
data_bases = cur.fetchall()

# Connect to the database
data_base = pymysql.connect(host='localhost', user='root', passwd='', database='demo')
cur = data_base.cursor()

# Define SQL queries to fetch data from the database
query_users = 'SELECT * FROM users'
query_bundles = 'SELECT * FROM bundles'

# Execute the queries and load the data into DataFrames
cur.execute(query_users)
users_df = pd.read_sql(query_users, data_base)
cur.execute(query_bundles)
bundle_df = pd.read_sql(query_bundles, data_base)

# Convert date columns to datetime format 
users_df['registration_date'] = pd.to_datetime(users_df['registration_date'])
users_df['subscription_date'] = pd.to_datetime(users_df['subscription_date'])
bundle_df['creation_date'] = pd.to_datetime(bundle_df['creation_date'])
st.sidebar.title('Electro Pi Data Dashboard')
sidebar_image = Image.open('logo.png')
st.sidebar.image(sidebar_image)
# Add a sidebar to navigate between pages
st.sidebar.title("Dashboard Pages")
page = st.sidebar.selectbox("Select a Page", ["User Analytics Dashboard", "User Learning and Completion Trends", "Admins, Coupons, and Employment Grants"])
    
if page == "User Analytics Dashboard":
        # question 1
        st.title("User Insights Across Registration, Subscriptions, and Demographics")
        st.subheader('1-The number of Registrated and Subscribed users daily, weekly, monthly and yearly ')

        # Create a radio to choose the time interval (daily, weekly, monthly, yearly) for both user data and bundle usage
        selected_interval = st.sidebar.radio("Select Time Interval", ["Daily", "Weekly", "Monthly", "Yearly"])

        # Define a function to calculate user counts based on the chosen time interval
        def calculate_users_by_time_interval(data, time_interval, column_name):
            if time_interval == "Daily":
                return data.groupby(data[column_name].dt.date).size()
            elif time_interval == "Weekly":
                return data.groupby(data[column_name].dt.strftime('%Y-%U')).size()
            elif time_interval == "Monthly":
                return data.groupby(data[column_name].dt.strftime('%Y-%m')).size()
            elif time_interval == "Yearly":
                return data.groupby(data[column_name].dt.year).size()

        # Calculate user counts based on the chosen time interval for user registration and subscription
        user_registration_counts = calculate_users_by_time_interval(users_df, selected_interval, 'registration_date')
        user_subscription_counts = calculate_users_by_time_interval(users_df, selected_interval, 'subscription_date')
        
        def create_color_list(data_series, default_color='blue', max_color='green', min_color='red'):
            colors = [default_color] * len(data_series)
            if not data_series.empty:
                max_value_index = data_series.index.get_loc(data_series.idxmax())
                min_value_index = data_series.index.get_loc(data_series.idxmin())

                colors[max_value_index] = max_color
                if max_value_index != min_value_index:
                    colors[min_value_index] = min_color

            return colors
        
        # Calculate max info for subscription and registration
        def calculate_max_info(data_series):
            if not data_series.empty:
                max_value = data_series.max()
                max_date = data_series.idxmax()  # Get the date/index of the maximum value
                return f"Date: {max_date}, Value: {max_value}"
            return ""

        subscription_max_info = calculate_max_info(user_subscription_counts)
        registration_max_info = calculate_max_info(user_registration_counts)

        # Display max info using Streamlit metrics
        col1, col2 = st.columns(2)
        col1.metric(label="Maximum Value and Date for Subscription", value=subscription_max_info, delta="Supsription" )
        col2.metric(label="Maximum Value and Date for Registration", value=registration_max_info, delta="Registration")
        
        style_metric_cards(background_color="#3F588C",border_left_color="#686664",border_color="#000000",box_shadow="#F71938")
 
        # Generate color lists for the charts
        subscription_colors = create_color_list(user_subscription_counts, default_color='blue', max_color='green', min_color='red')
        registration_colors = create_color_list(user_registration_counts, default_color='blue', max_color='green', min_color='red')

        # Example for creating a bar chart with custom annotations for max/min
        def create_bar_chart_with_annotations(data_series, colors, title,  width=1500, height=600):
            # Create the bar chart
            fig = go.Figure(data=[go.Bar(
                x=data_series.index, 
                y=data_series.values, 
                marker_color=colors  # Use the custom colors
            )])

            # Update layout for custom annotations
            fig.update_layout(
                title=title,
                xaxis_title=f"Time ({selected_interval})",
                yaxis_title='User Count',
                annotations=[
                    dict(
                        x=0.5, y=1.1, xref='paper', yref='paper',
                        text='Max: Green, Min: Red', showarrow=False,
                        font=dict(size=12, color="black"),
                        bgcolor='white', bordercolor='black', borderwidth=1,
                        xanchor='center', yanchor='bottom'
                    )
                ]
            )

            return fig
        
       
        # Use this function to create your charts
        
        fig_registration = create_bar_chart_with_annotations(
            user_registration_counts,
            create_color_list(user_registration_counts, default_color='blue', max_color='green', min_color='red'),
            "User Registration Count"
        )
        fig_registration.update_layout(
                paper_bgcolor="#3F588C",
                plot_bgcolor='black',
                xaxis=dict(showgrid=False),  # Remove gridlines for x-axis
                yaxis=dict(showgrid=False)   # Remove gridlines for y-axis

            )
        

        st.plotly_chart(fig_registration, use_container_width=True)
        

        fig_subscription = create_bar_chart_with_annotations(
            user_subscription_counts,
            create_color_list(user_subscription_counts, default_color='blue', max_color='green', min_color='red'),
            "User Subscription Count"
        )
        fig_subscription.update_layout(
                #paper_bgcolor="#3F588C",
                paper_bgcolor="#3F588C",
                plot_bgcolor='black',
                xaxis=dict(showgrid=False),  # Remove gridlines for x-axis
                yaxis=dict(showgrid=False)   # Remove gridlines for y-axis

            )
        st.plotly_chart(fig_subscription, use_container_width=True)
            
        col3, padding, col4 = st.columns((17,1,8))
        
        with col4:
            def calculate_user_counts_by_bundle(data, time_interval):
                if time_interval == "Daily":
                    return data.groupby([data['creation_date'].dt.date, 'bundle_name']).size().unstack(fill_value=0)
                elif time_interval == "Weekly":
                    return data.groupby([data['creation_date'].dt.strftime('%Y-%U'), 'bundle_name']).size().unstack(fill_value=0)
                elif time_interval == "Monthly":
                    return data.groupby([data['creation_date'].dt.strftime('%Y-%m'), 'bundle_name']).size().unstack(fill_value=0)
                elif time_interval == "Yearly":
                    return data.groupby([data['creation_date'].dt.year, 'bundle_name']).size().unstack(fill_value=0)

            # Calculate user counts by bundle and time interval for bundle usage
            user_counts_daily = calculate_user_counts_by_bundle(bundle_df, selected_interval)

            # Calculate total subscriptions
            total_subscriptions = bundle_df['bundle_name'].count()
            st.metric("Total Subscriptions", total_subscriptions)

            # Most and least popular bundles
            most_popular_bundle = user_counts_daily.sum().idxmax()
            least_popular_bundle = user_counts_daily.sum().idxmin()
            st.metric("Most Popular Bundle", most_popular_bundle)
            st.metric("Least Popular Bundle", least_popular_bundle)
            style_metric_cards(background_color="#3F588C",border_left_color="#686664",border_color="#000000",box_shadow="#F71938")
            
            
        with col3:  
            def create_bundle_usage_chart(user_counts, time_interval):
                
                fig = px.bar(
                    user_counts.reset_index(),
                    x='creation_date',
                    y=user_counts.columns,
                    labels={'creation_date': f"Time ({time_interval})", 'value': 'User Count'},
                )
                fig.update_layout(xaxis_tickangle=-45, title=f"Bundle Usage ({time_interval})")
                return fig

            # Display the user counts by bundle as a Plotly bar chart for bundle usage
            st.subheader('2-User Subscriptions by Bundle daily, weekly, monthly and yearly')

            # Create the figure with the specified interval
            fig = create_bundle_usage_chart(user_counts_daily, selected_interval)

            # Update the layout of the created figure
            fig.update_layout(
                paper_bgcolor="#3F588C",
                plot_bgcolor='black',
                xaxis=dict(showgrid=False),  # Remove gridlines for x-axis
                yaxis=dict(showgrid=False)   # Remove gridlines for y-axis
            )

            # Display the figure in Streamlit
            st.plotly_chart(fig, use_container_width=True)

            

        col5, padding, col6 = st.columns((15,1,7))   
       
        with col5:
            

            color_map = {
                'Student': 'yellow',  # Replace 'Study Degree 1' with the actual name of the study degree
                'Graduated': 'blue',    # Replace with other study degrees and colors as needed
                # Add more mappings for each study degree
            }
            st.subheader('3-The number of users grouped by age, and study degree')
            # Group the data by 'age' and 'study_degree' and count the number of users in each group
            grouped_data = users_df.groupby(['age', 'study_degree']).size().reset_index(name='User Count')

            # Create a bar chart using Plotly Express
            fig = px.bar(grouped_data, x='age', y='User Count', color='study_degree', barmode='group',
                         labels={'age': 'Age', 'User Count': 'Number of Users', 'study_degree': 'Study Degree'},
                         title='Number of Users by Age and Study Degree',
                        color_discrete_map=color_map)
            fig.update_xaxes(dtick=5)

            fig.update_layout(
                paper_bgcolor="#3F588C",
                plot_bgcolor='black',
                xaxis=dict(showgrid=False),
                yaxis=dict(showgrid=False)
                
            )

            st.plotly_chart(fig, use_container_width=True)

           
         

        with col6:
        
            # Calculate the counts of students and graduates
            count_student = users_df[users_df['study_degree'] == 'Student'].shape[0]
            count_graduated = users_df[users_df['study_degree'] == 'Graduated'].shape[0]

            # Calculate other metrics
            total_users = users_df.shape[0]
            average_age = users_df['age'].mean()
            

            # Calculate the percentage of students and graduates
            percentage_student = (count_student / total_users) * 100
            percentage_graduated = (count_graduated / total_users) * 100

            # Display metrics using st.metric
            st.subheader('Metrics')
            st.metric(label='Total Users', value=total_users)
            st.metric(label='Average Age', value=round(average_age))  # Rounding average age to integer
              
            st.metric(label='Percentage of Students', value=f'{percentage_student:.2f}%')
            st.metric(label='Percentage of Graduates', value=f'{percentage_graduated:.2f}%')

            

            query_9 = '''
                SELECT age, study_degree, COUNT(*) as user_count
                FROM demo.users
                GROUP BY age, study_degree
            '''

            # Execute the query and load the data into the DataFrame
            cur.execute(query_9)
            age_degree_count_df = pd.read_sql(query_9, data_base)
            
            #st.subheader("User Count by Age and Study Degree")
            #st.dataframe(age_degree_count_df)
            # Create an expander to show/hide the dataframe
            with st.expander("Click to view User Count by Age and Study Degree"):
                st.dataframe(age_degree_count_df)

                  

elif  page== "User Learning and Completion Trends":
    
    st.title('Comprehensive Learner Engagement and Achievement')  
     #question4   
    st.subheader('1-All Users Course Activity')
    # Load the user lesson history data
    query = 'SELECT * FROM user_lesson_history'
    df_lesson_history = pd.read_sql(query, data_base)

    # Convert 'last_viewed' column to datetime
    df_lesson_history['last_viewed'] = pd.to_datetime(df_lesson_history['last_viewed'])

    # Define the current date
    current_date = datetime.now()

    # Filter lessons completed during the current week, month, and year
    this_week = df_lesson_history[df_lesson_history['last_viewed'].dt.strftime('%Y-%U') == current_date.strftime('%Y-%U')]
    this_month = df_lesson_history[df_lesson_history['last_viewed'].dt.strftime('%Y-%m') == current_date.strftime('%Y-%m')]
    this_year = df_lesson_history[df_lesson_history['last_viewed'].dt.strftime('%Y') == current_date.strftime('%Y')]

    # Group by user_id to count the number of currently learning courses and completed courses
    learning_courses_count = df_lesson_history[df_lesson_history['count'] > 0].groupby('user_id')['course_id'].nunique()
    completed_courses_count_week = this_week[this_week['count'] > 0].groupby('user_id')['course_id'].nunique()
    completed_courses_count_month = this_month[this_month['count'] > 0].groupby('user_id')['course_id'].nunique()
    completed_courses_count_year = this_year[this_year['count'] > 0].groupby('user_id')['course_id'].nunique()

    # Merge the counts into a single DataFrame
    user_counts = pd.DataFrame({
        'Currently Learning Courses': learning_courses_count,
        'Completed Courses This Week': completed_courses_count_week,
        'Completed Courses This Month': completed_courses_count_month,
        'Completed Courses This Year': completed_courses_count_year
    }).fillna(0).astype(int)
    
    col1, col2, col3  = st.columns(3)
    most_popular_course_id = df_lesson_history.groupby('course_id').size().idxmax()
    most_popular_course_count = df_lesson_history.groupby('course_id').size().max()
    col1.metric(label="Most Popular Course ID", value=most_popular_course_id)

    
    
    total_courses = df_lesson_history['course_id'].nunique()
    col2.metric(label="Total Courses", value=total_courses)
    total_users = df_lesson_history['user_id'].nunique()
    col3.metric(label="Total Users", value=total_users)

    col1, col2, col3  = st.columns(3)
    active_users = df_lesson_history[df_lesson_history['last_viewed'] > pd.Timestamp('now') - pd.Timedelta(days=30)]['user_id'].nunique()
    col1.metric(label="Active Users the last 30 days", value=active_users)
    total_completed_courses = df_lesson_history[df_lesson_history['count'] > 0].shape[0]
    col2.metric(label="Total Completed Courses", value=total_completed_courses)
    
    

    # Sidebar filter for time frame selection
    time_frame = st.sidebar.radio(
        "Select Time Frame for Course Completion:",
        ('This Week', 'This Month', 'This Year')
    )

    # Calculate the number of courses completed based on the selected time frame
    if time_frame == 'This Week':
        completed_courses_time_frame = this_week[this_week['count'] > 0].shape[0]
    elif time_frame == 'This Month':
        completed_courses_time_frame = this_month[this_month['count'] > 0].shape[0]
    else:  # 'This Year'
        completed_courses_time_frame = this_year[this_year['count'] > 0].shape[0]

    # Display the metric
    col3.metric(label=f"Courses Completed {time_frame}", value=completed_courses_time_frame)
    
    style_metric_cards(background_color="#3F588C",border_left_color="#686664",border_color="#000000",box_shadow="#F71938")

    # Display the table
    sorted_user_counts = user_counts.sort_values(by='Currently Learning Courses', ascending=False)
    
    with st.expander("User Course Activity sorted by Currently Learning Courses"):
        st.dataframe(sorted_user_counts)
    
    sorted_user_counts_week = user_counts.sort_values(by='Completed Courses This Week', ascending=False)

    # Display the table in an expander
    with st.expander("User Course Activity sorted by Completed Courses This Week"):
        st.dataframe(sorted_user_counts_week)
        
    sorted_user_counts_month = user_counts.sort_values(by='Completed Courses This Month', ascending=False)
        
    with st.expander("User Course Activity sorted by Completed Courses This Month"):
        st.dataframe(sorted_user_counts_month)
    
    sorted_user_counts_year = user_counts.sort_values(by='Completed Courses This Year', ascending=False)
        
    with st.expander("User Course Activity sorted by Completed Courses This Year"):
        st.dataframe(sorted_user_counts_year)
        
    
        

    # Rank users in each category
    user_counts['Rank This Month'] = user_counts['Completed Courses This Month'].rank(ascending=False, method='min')
    user_counts['Rank This Week'] = user_counts['Completed Courses This Week'].rank(ascending=False, method='min')
    user_counts['Rank This Year'] = user_counts['Completed Courses This Year'].rank(ascending=False, method='min')

    # Calculate a combined rank (you can tweak this formula as needed)
    user_counts['Combined Rank'] = user_counts['Rank This Month'] + user_counts['Rank This Week'] + user_counts['Rank This Year']

    # Sort by the combined rank to find top users across all categories
    top_users_across_categories = user_counts.sort_values(by='Combined Rank').head(10)  # You can adjust the number of top users

    # Display the top users in an expander
    with st.expander("Top 10 Users Across All Categories"):
        st.dataframe(top_users_across_categories)


    # Create a Streamlit input field to enter the user ID
    user_id = st.text_input("Enter User ID from the table above to get information about each user:")

    # Check if user_id is provided and valid
    if user_id:
        try:
            user_id = int(user_id)  # Convert the input to an integer

            # Filter the data for the specified user_id
            user_data_learning = df_lesson_history[df_lesson_history['user_id'] == user_id]
            user_data_week = this_week[this_week['user_id'] == user_id]
            user_data_month = this_month[this_month['user_id'] == user_id]
            user_data_year = this_year[this_year['user_id'] == user_id]

            # Count the number of unique courses for each category
            learning_courses_count = user_data_learning[user_data_learning['count'] > 0]['course_id'].nunique()
            completed_courses_count_week = user_data_week[user_data_week['count'] > 0]['course_id'].nunique()
            completed_courses_count_month = user_data_month[user_data_month['count'] > 0]['course_id'].nunique()
            completed_courses_count_year = user_data_year[user_data_year['count'] > 0]['course_id'].nunique()

            # Define colors for each category
            colors = ['blue', 'green', 'orange', 'red']

            # Create a Plotly bar chart
            data = {
                'Category': ['Currently Learning Courses', 'Completed Courses This Week', 'Completed Courses This Month', 'Completed Courses This Year'],
                'Course Count': [learning_courses_count, completed_courses_count_week, completed_courses_count_month, completed_courses_count_year]
            }

            fig = px.bar(data, x='Category', y='Course Count', title=f'Course Activity for User {user_id}', color='Category', color_discrete_sequence=colors)
            fig.update_layout(width=1000, height=500)
            # Update the layout of the created figure
            fig.update_layout(
                paper_bgcolor="#3F588C",
                plot_bgcolor='black',
                xaxis=dict(showgrid=False),  # Remove gridlines for x-axis
                yaxis=dict(showgrid=False)   # Remove gridlines for y-axis
            )

            # Display the figure in Streamlit
            st.plotly_chart(fig, use_container_width=True)

            #st.plotly_chart(fig)
        except ValueError:
            st.warning("Please enter a valid User ID (numeric value).")

    #question3
    
    # Load the data
    
    query = 'SELECT * FROM users_on_ai_initiative'
    df_10kai = pd.read_sql(query, data_base)

    st.subheader("2-All 10k AI Initiative Users Learning Progress")

    # Calculate the number of completed courses for each user
    completed_courses_count = df_10kai.groupby("user_id")["course_id"].count().reset_index()
    completed_courses_count.columns = ["user_id", "completed_courses_count"]

    # Merge the completed courses count back into the original DataFrame
    df_10kai = pd.merge(df_10kai, completed_courses_count, on="user_id", how="left")

    # Filter the DataFrame to include only users in the 10k AI initiative (10k_AI_initiative == 1)
    filtered_df = df_10kai[df_10kai["10k_AI_initiative"] == 1]

    # Group by user_id to calculate the number of completed courses for each user
    completed_courses_count = filtered_df.groupby("user_id")["course_id"].count().reset_index()
    completed_courses_count.columns = ["user_id", "Completed Courses"]

    # Get the information of the last completed course for each user
    last_completion_info = filtered_df.groupby("user_id").apply(lambda x: x.nlargest(1, "completion_date")).reset_index(drop=True)
    last_completion_info = last_completion_info[["user_id", "completion_date", "course_degree"]]

    # Merge the completed courses count and last completion information into a single DataFrame
    table_data = pd.merge(completed_courses_count, last_completion_info, on="user_id")

    # Sort the table by "Completed Courses" in descending order
    table_data = table_data.sort_values(by="Completed Courses", ascending=False)
    

    # Assuming you have already calculated the metrics
    total_users = len(table_data)
    average_completed_courses = round(table_data["Completed Courses"].mean())
    most_common_degree = table_data["course_degree"].mode().iloc[0]
    # Get the ID of the person with the highest number of completed courses
    user_with_highest_completion = table_data.loc[table_data['Completed Courses'].idxmax()]['user_id']

    # Calculate the percentage of users with a degree
    users_with_degree = table_data[table_data["course_degree"].notnull()].shape[0]
    percentage_users_with_degree = (users_with_degree / total_users) * 100
    
    percentage_users_without_degree = (table_data["course_degree"].isnull().sum() / total_users) * 100
    
    # Calculate the percentage of users without a degree
    with st.container():
        # First row with three columns
        col1, col2, col3 = st.columns(3)

        # Display metrics in each column
        col1.metric(label='Total Users', value=total_users)
        col2.metric(label='Average Completed Courses per User', value=average_completed_courses)
        col3.metric(label='Most Common Degree', value=most_common_degree)

    with st.container():
        # Second row with three columns
        col4, col5, col6 = st.columns(3)

        # Display metrics in each column
        col4.metric(label='User ID Highest number of Completed Courses', value=user_with_highest_completion)
        col5.metric(label='Users with at least one completed coures (%)', value=f'{percentage_users_with_degree:.2f}%')
        col6.metric(label='Users without any completed courses (%)', value=f'{percentage_users_without_degree:.2f}%')

    style_metric_cards(background_color="#3F588C",border_left_color="#686664",border_color="#000000",box_shadow="#F71938")    

    # Create an expander to make the table collapsible
    with st.expander("User Information in the 10k AI Initiative (Sorted by Completed Courses)"):
        table_data['course_degree'] = table_data['course_degree'].apply(lambda x: f'{x:.0f}')

        # Display the table
        st.dataframe(table_data)

    # Get user input for the user ID
    user_id_input = st.text_input("Enter User ID from the table above:")

    # Check if user_id_input is provided and valid
    if user_id_input:
        try:
            user_id_input = int(user_id_input)  # Convert the input to an integer

            # Filter the data for the specified user_id_input
            user_data_input = table_data[table_data['user_id'] == user_id_input]

            # Check if the user_data_input is not empty
            if not user_data_input.empty:

                # Create a Plotly bar chart
                fig = px.bar(user_data_input, x='completion_date', y='Completed Courses',
                             title=f'User {user_id_input} Completed Courses Progress',
                             labels={'completion_date': 'Completion Date', 'Completed Courses': 'Completed Courses'},
                             color='course_degree',color_discrete_sequence=['orange'])
                fig.update_layout(
                    paper_bgcolor="#3F588C",
                    plot_bgcolor='black',
                    xaxis=dict(showgrid=False),  # Remove gridlines for x-axis
                    yaxis=dict(showgrid=False)   # Remove gridlines for y-axis
                )

                # Display the chart
                st.plotly_chart(fig)

            else:
                st.warning("User ID not found in the table above.")
        except ValueError:
            st.warning("Please enter a valid User ID (numeric value).")
    
    #Question5
    #user information
    col1, padding, col2 = st.columns((10,1,10))
    with col1:
        np.random.choice(users_df['user_id'].values)
        def random_idd(x):
            return np.random.choice(users_df['user_id'].values)
        bundle_df['user_id']=bundle_df['user_id'].apply(random_idd)

        user_bundel_df=pd.merge(users_df, bundle_df,on='user_id')

        query = 'SELECT * FROM user_courses'
        user_courses_df = pd.read_sql(query, data_base)
        user_bundel_courses=pd.merge(user_bundel_df,user_courses_df[['user_id','course_id']],on='user_id' )

        query = 'SELECT * FROM courses'
        courses_df = pd.read_sql(query, data_base)
        df=pd.merge(user_bundel_courses,courses_df, on='course_id')

        query = 'SELECT * FROM user_lesson_history WHERE degree>0'
        lesson_df = pd.read_sql(query, data_base)
        final_df=pd.merge(df,lesson_df[['course_id' ,'chapter_id' ,'lesson_id' ,'count' ,'degree']],on='course_id')

        final_df.drop(['coupon','subscription_date'], axis=1, inplace=True)
        data=final_df['user_id'].to_list()

        
        st.subheader('3-Individual Learner Profile')

        # Get unique user IDs for the select box
        unique_user_ids = final_df['user_id'].unique()

        # User selection dropdown
        selected_user_from_select = st.selectbox("Select a User", unique_user_ids)

        # User input field, providing an alternative to the select box
        selected_user_from_input = st.text_input("Or enter a User ID here", "")

        # Determine which selection to use, prioritizing manual input if provided
        if selected_user_from_input:
            try:
                user_id = int(selected_user_from_input)
            except ValueError:
                st.error("Please enter a valid User ID.")
        else:
            user_id = selected_user_from_select



        # Data processing and visualization based on user_id
        if user_id in final_df['user_id'].values:
            user_data = final_df[final_df['user_id'] == user_id]                     
            # Display Metrics
            st.subheader('User Profile Important Metrics')
            
            # Define columns for displaying metrics
            st.metric("Part of 10k AI Initiative", "Yes" if user_data['10k_AI_initiative'].iloc[0] == 1 else "No") 
            st.metric("User Level", user_data['level'].iloc[0])
            #st.metric("Age", user_data['age'].iloc[0])
            #st.metric("Study Degree", user_data['study_degree'].iloc[0])
            #st.metric("Subscription Status", "Subscribed" if user_data['subscribed'].iloc[0] == 1 else "Not Subscribed")


        else:
            st.write("User Not Found")
            
    with col2:
                st.table(user_data.iloc[0])
    
        
elif  page== "Admins, Coupons, and Employment Grants":
    st.title('Integrated Insights Hub: Admins, Coupons, and Employment Grants')    
    col1, padding, col2 = st.columns((10,1,10))
   
    with col1:
          #question 10 
        # Define SQL query
        query_10 = '''SELECT user_id, status, application_date FROM demo.users_employment_grant;'''


        # Execute the query and load the data into the DataFrame
        cur.execute(query_10)
        grant_status_df = pd.read_sql(query_10, data_base)
        st.subheader("1-User Employment Grant Status")
        
        grant_status_df = grant_status_df.fillna('Undetermined')
        gs_df = grant_status_df['status'].value_counts().reset_index()
        gs_df.columns = ['Status', 'Count']

        # Create a pie chart using Plotly Express
        fig = px.pie(gs_df, values='Count', names='Status', title="User Employment Grant Status Distribution")

        # Update the layout of the created figure
            
        fig.update_layout(
            paper_bgcolor='#3F588C',  
            margin=dict(l=20, r=20, t=40, b=40), 
            showlegend=True,
            plot_bgcolor = 'black'
            )  # Display the legend
        # Display the pie chart in Streamlit
        st.plotly_chart(fig,use_container_width=True )      

        
    with col2:
        with st.expander("View Full Employment Grant Status Data"):
            
            st.write(grant_status_df)
       
        user_id_input = st.text_input("Enter User ID:", "")
        
        
        
        # Display employment grant status for the entered user_id
        if user_id_input:
            user_id_input = int(user_id_input)  # Assuming user_id is an integer
            user_info = grant_status_df[grant_status_df['user_id'] == user_id_input]


            if not user_info.empty:
                user_status = user_info['status'].values[0]
                # Retrieve application date and convert it to a formatted string
                application_date = user_info['application_date'].values[0]
                if isinstance(application_date, (datetime, pd.Timestamp)):
                    formatted_date = application_date.strftime("%Y-%m-%d")  # Format: Year-Month-Day
                else:
                    formatted_date = str(application_date)  # Convert to string if not a datetime object

                st.subheader(f"Employment Grant Information for User {user_id_input}:")
                st.metric(label='Employment Grant Status', value=user_status)
                st.metric(label='Application Date', value=formatted_date)  # Display formatted application date
            else:
                st.warning(f"No information found for User ID: {user_id_input}")


        # Count the values in the 'status' column
        grant_status_df = grant_status_df.fillna('Undetermined')
        gs_df = grant_status_df['status'].value_counts().reset_index()
        gs_df.columns = ['Status', 'Count']

        # Display the status count
        st.subheader("Employment Grant Status Count:")
        st.write(gs_df)



        
     
    col3, padding, col4 = st.columns((10,1,3))
    with col3:
                    
        #question 8 
        query = 'SELECT * FROM copons'
        df_copons = pd.read_sql(query, data_base)
        st.subheader('2-Coupon Usage Statistics')

        # Calculate the top 10 coupons by the number of users
        top_7_coupons = df_copons.groupby('copon_code')['users'].sum().nlargest(7)

        # Create a bar chart
        fig = px.bar(top_7_coupons, x=top_7_coupons.index, y=top_7_coupons.values, labels={'x': 'Coupon Code', 'y': 'Number of Users'},color=top_7_coupons.index)
        fig.update_layout(title='Top 7 Coupons Used by Number of Users')
        fig.update_layout(width=800, height=500)
        # Display the chart in Streamlit
        fig.update_layout(
                paper_bgcolor="#3F588C",
                plot_bgcolor='black',
                xaxis=dict(showgrid=False),  # Remove gridlines for x-axis
                yaxis=dict(showgrid=False)   # Remove gridlines for y-axis
            )
        st.plotly_chart(fig)
        
    with col4:
        # User selection dropdown for choosing a coupon
        selected_coupon = st.selectbox("Select a Coupon", df_copons['copon_code'].unique())

        # Calculate and display the number of actual users who used the selected coupon
        if selected_coupon:
            st.write(f"Number of Users for Coupon '{selected_coupon}'")

            # Filter data for the selected coupon code
            selected_coupon_data = df_copons[df_copons['copon_code'] == selected_coupon]

            # Calculate the number of unique users for the selected coupon
            num_users = selected_coupon_data['users'].sum()
            st.write(f"Number of Users: {num_users}")
            st.metric(label='Number of Users', value=num_users)
            style_metric_cards(background_color="#3F588C",border_left_color="#686664",border_color="#000000",box_shadow="#F71938")
            
    col3, padding, col4 = st.columns((5,1,5))
    with col3:
        #question number 6 

        # Database query to fetch data
        query = 'SELECT * FROM capstone_evaluation_history'
        df_capstone_evaluations = pd.read_sql(query, data_base)

        # Convert 'evaluation_date' to datetime
        df_capstone_evaluations['evaluation_date'] = pd.to_datetime(df_capstone_evaluations['evaluation_date']).dt.date

        # Streamlit interface setup
        st.subheader("3-Admin Capstone Evaluation")
        # Add radio button for time period selection
        
        time_filter = st.sidebar.radio("Select Time Period for Admin Capstone Evaluation", ["All Time", "Today", "This Week", "This Month"])

        # Apply time filter
        today = datetime.now().date()
        if time_filter == "Today":
            filtered_data = df_capstone_evaluations[df_capstone_evaluations['evaluation_date'] == today]
        elif time_filter == "This Week":
            start_week = today - timedelta(days=today.weekday())
            end_week = start_week + timedelta(days=6)
            filtered_data = df_capstone_evaluations[(df_capstone_evaluations['evaluation_date'] >= start_week) & 
                                                    (df_capstone_evaluations['evaluation_date'] <= end_week)]
        elif time_filter == "This Month":
            start_month = today.replace(day=1)
            end_month = today.replace(month=today.month % 12 + 1, day=1) - timedelta(days=1)
            filtered_data = df_capstone_evaluations[(df_capstone_evaluations['evaluation_date'] >= start_month) & 
                                                    (df_capstone_evaluations['evaluation_date'] <= end_month)]
        else:
            # Use the entire range if "All Time" is selected
            start_date = df_capstone_evaluations['evaluation_date'].min()
            end_date = df_capstone_evaluations['evaluation_date'].max()
            filtered_data = df_capstone_evaluations[(df_capstone_evaluations['evaluation_date'] >= start_date) & 
                                                    (df_capstone_evaluations['evaluation_date'] <= end_date)]

        # Aggregate data by Admin ID
        admin_evaluation_counts = filtered_data.groupby('admin_id').size().reset_index(name='Evaluation Count')

        # Sort the DataFrame by 'Evaluation Count' in descending order
        admin_evaluation_counts_sorted = admin_evaluation_counts.sort_values(by='Evaluation Count', ascending=False)

        # Metrics for highest and lowest count
        if not admin_evaluation_counts_sorted.empty:
            highest_count_admin = admin_evaluation_counts_sorted.iloc[0]
            lowest_count_admin = admin_evaluation_counts_sorted.iloc[-1]
            st.metric("Highest Evaluation Count", f"Admin ID {highest_count_admin['admin_id']}")
            st.metric("Lowest Evaluation Count", f"Admin ID {lowest_count_admin['admin_id']}")

        # User input to get count for a specific admin ID
        user_admin_id = st.sidebar.number_input("Enter Admin ID to see the evaluation count:", min_value=0, format='%d')
        if user_admin_id:
            user_admin_count = admin_evaluation_counts_sorted[admin_evaluation_counts_sorted['admin_id'] == user_admin_id]['Evaluation Count'].sum()
            st.sidebar.metric(f"Evaluation Count for Admin ID {user_admin_id}", user_admin_count)

        # Ensure admin_id is treated as a categorical variable
        admin_evaluation_counts_sorted['admin_id'] = admin_evaluation_counts_sorted['admin_id'].astype(str)

        # Visualize the data with different colors for each admin
        fig = px.bar(admin_evaluation_counts_sorted, x='admin_id', y='Evaluation Count', title='Capstone Evaluations per Admin', text='Evaluation Count', color='admin_id')
        fig.update_xaxes(type='category')  # Force x-axis to be treated as a category
        # Adjust the size of the chart
        fig.update_layout(
            height=500,  # Adjust the height as needed
            width=500    # Adjust the width as needed
        )
        fig.update_layout(
                paper_bgcolor="#3F588C",
                plot_bgcolor='black',
                xaxis=dict(showgrid=False),  # Remove gridlines for x-axis
                yaxis=dict(showgrid=False)   # Remove gridlines for y-axis
            )

        st.plotly_chart(fig)

        



    with col4:
        
        st.subheader("4-Capstone Evaluation History")
        # Build a dashboard to show each user’s capstone and the evaluation history of this capstone.
        query = 'SELECT * FROM capstone_evaluation_history'
        df_capstone_evaluation_history = pd.read_sql(query, data_base)

        # Get unique user IDs for the select box
        unique_user_ids = df_capstone_evaluation_history['user_id'].unique()

        # User selection dropdown
        selected_user_from_select = st.selectbox("Select a User", unique_user_ids)

        # User input field, providing an alternative to the select box
        selected_user_from_input = st.text_input("...or enter a User ID here", "")

        # Determine which selection to use, prioritizing manual input if provided
        selected_user = selected_user_from_input or selected_user_from_select

        # Convert the selected_user to the correct type, e.g., int or str, as needed
        try:
            selected_user = int(selected_user)
        except ValueError:
            if selected_user_from_input:  # only show an error if the input field was used
                st.error("Please enter a valid User ID.")
                st.stop()

        # Filter capstone information for the selected user
        user_capstones = df_capstone_evaluation_history[df_capstone_evaluation_history['user_id'] == selected_user]

        # Display capstone information if any
        if not user_capstones.empty:
            st.write("Capstone Information for User:", selected_user)
            st.write(user_capstones)
            # Calculate metrics
            average_score = user_capstones['degree'].mean()
            total_evaluations = user_capstones.shape[0]
            highest_score = user_capstones['degree'].max()
            lowest_score = user_capstones['degree'].min()
            last_evaluation_date = user_capstones['evaluation_date'].max()

            # Define columns for displaying metrics side by side
            col1, col2, col3 = st.columns(3)

            col1.metric("Average Score", f"{average_score:.2f}")
            col2.metric("Lowest Score", lowest_score)
            col3.metric("Highest Score", highest_score)
            
            col1, col2 = st.columns(2)
            
            
            col1.metric("Total Evaluations", total_evaluations)
            col2.metric("Last Evaluation Date", last_evaluation_date.strftime('%Y-%m-%d'))

            
        else:
            st.write("No capstone information available for the selected user.")
            
            
    
   

Overwriting dashboard.py


In [2]:
!streamlit run dashboard.py

^C
